In [1]:
import itertools
import logging
import pathlib
import sys
from typing import Optional

import numpy as np
import pandas as pd
import toml
from copairs.map import run_pipeline
from pycytominer import feature_select

# imports src
sys.path.append("../")
from src import utils

# setting up logger
logging.basicConfig(
    filename="map_analysis_testing.log",
    level=logging.DEBUG,
    format="%(levelname)s:%(asctime)s:%(name)s:%(message)s",
)

## Helper functions
Set of helper functions to help out throughout the notebook

In [2]:
## Helper function


def shuffle_meta_labels(
    dataset: pd.DataFrame, target_col: str, seed: Optional[int] = 0
) -> pd.DataFrame:
    """shuffles labels or values within a single selected column

    Parameters
    ----------
    dataset : pd.DataFrame
        dataframe containing the dataset

    target_col : str
        Column to select in order to conduct the shuffling

    seed : int
        setting random seed

    Returns
    -------
    pd.DataFrame
        shuffled dataset

    Raises
    ------
    TypeError
        raised if incorrect types are provided
    """
    # setting seed
    np.random.seed(seed)

    # type checking
    if not isinstance(target_col, str):
        raise TypeError("'target_col' must be a string type")
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("'dataset' must be a pandas dataframe")

    # selecting column, shuffle values within column, add to dataframe
    dataset[target_col] = np.random.permutation(dataset[target_col].values)
    return dataset


def shuffle_features(feature_vals: np.array, seed: Optional[int] = 0) -> np.array:
    """suffles all values within feature space

    Parameters
    ----------
    feature_vals : np.array
        shuffled

    seed : Optional[int]
        setting random seed

    Returns
    -------
    np.array
        Returns shuffled values within the feature space

    Raisespairs(sameby=pos_s
    TypeError
        Raised if a numpy array is not provided
    """
    # setting seed
    np.random.seed(seed)

    # shuffle given array
    if not isinstance(feature_vals, np.ndarray):
        raise TypeError("'feature_vals' must be a numpy array")
    if feature_vals.ndim != 2:
        raise TypeError("'feature_vals' must be a 2x2 matrix")

    # creating a copy for feature vales to prevent overwriting of global variables
    feature_vals = np.copy(feature_vals)

    # shuffling feature space
    n_cols = feature_vals.shape[1]
    for col_idx in range(0, n_cols):
        # selecting column, shuffle, and update:
        feature_vals[:, col_idx] = np.random.permutation(feature_vals[:, col_idx])

    return feature_vals

## Setting up Paths and loading data

In [3]:
# load in the treatment groups
ground_truth = pathlib.Path(
    "../../4.sc_Morphology_Neural_Network_MLP_Model/MLP_utils/ground_truth.toml"
).resolve(strict=True)
# load in the ground truth
ground_truth = toml.load(ground_truth)
apoptosis_ground_truth = ground_truth["Apoptosis"]["apoptosis_groups_list"]
pyroptosis_ground_truth = ground_truth["Pyroptosis"]["pyroptosis_groups_list"]
control_ground_truth = ground_truth["Healthy"]["healthy_groups_list"]

In [4]:
single_cell_data = pathlib.Path(
    f"../../data/PBMC_preprocessed_sc_norm_aggregated.parquet"
).resolve(strict=True)
df = pd.read_parquet(single_cell_data)

In [5]:
# out paths
map_out_dir = pathlib.Path("../data/processed/mAP_scores/morphology/")
map_out_dir.mkdir(exist_ok=True, parents=True)

# regular data output
# saving to csv
regular_feat_map_path = pathlib.Path(map_out_dir / "mAP_scores_regular_class.csv")

# shuffled data output
shuffled_feat_map_path = pathlib.Path(map_out_dir / "mAP_scores_shuffled_class.csv")

# shuffled feature space output
shuffled_feat_space_map_path = pathlib.Path(
    map_out_dir / "mAP_scores_shuffled_feature_space_class.csv"
)

### Clean up data

In [6]:
# add apoptosis, pyroptosis and healthy columns to dataframe
df["Apoptosis"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in apoptosis_ground_truth,
    axis=1,
)
df["Pyroptosis"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in pyroptosis_ground_truth,
    axis=1,
)
df["Control"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in control_ground_truth,
    axis=1,
)

# merge apoptosis, pyroptosis, and healthy columns into one column
df["Metadata_labels"] = df.apply(
    lambda row: "Apoptosis"
    if row["Apoptosis"]
    else "Pyroptosis"
    if row["Pyroptosis"]
    else "Control",
    axis=1,
)
# # drop apoptosis, pyroptosis, and healthy columns
df.drop(columns=["Apoptosis", "Pyroptosis", "Control"], inplace=True)
df.drop(columns=["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"], inplace=True)

In [7]:
# output directories
map_out_dir = pathlib.Path("../data/processed/mAP_scores/")
map_out_dir.mkdir(parents=True, exist_ok=True)

### mAP Pipeline Parameters

The null size needs to be determined for the mAP pipeline. This is the size of the null class that is used to determine the mAP score.

In [8]:
tmp = (
    df.groupby(["Metadata_labels"])
    .count()
    .reset_index()[["Metadata_Well", "Metadata_labels"]]
)
# get the Pyroptosis number of Metadata_Well
Pyroptosis_count = tmp[tmp["Metadata_labels"] == "Pyroptosis"]["Metadata_Well"].values[
    0
]
Pyroptosis_count

65

In [9]:
df["Metadata_Well"].unique()

array(['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10',
       'B11', 'B12', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08',
       'C09', 'C10', 'C11', 'C12', 'D02', 'D03', 'D04', 'D05', 'D06',
       'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'E02', 'E03', 'E04',
       'E05', 'E06', 'E07', 'E08', 'E09', 'E10', 'E11', 'E12', 'F02',
       'F03', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11',
       'F12', 'G02', 'G03', 'G04', 'G05', 'G06', 'G07', 'G08', 'G09',
       'G10', 'G11', 'G12', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07',
       'H08', 'H09', 'H10', 'H11', 'H12', 'I02', 'I03', 'I04', 'I05',
       'I06', 'I07', 'I08', 'I09', 'I10', 'I11', 'I12', 'J02', 'J03',
       'J04', 'J05', 'J06', 'J07', 'J08', 'J09', 'J10', 'J11', 'J12',
       'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08', 'K09', 'K10',
       'K11', 'K12', 'L02', 'L03', 'L04', 'L05', 'L06', 'L07', 'L08',
       'L09', 'L10', 'L11', 'L12', 'M02', 'M03', 'M04', 'M05', 'M06',
       'M07', 'M08',

In [10]:
df["Metadata_labels"].unique()

array(['Pyroptosis', 'Control', 'Apoptosis'], dtype=object)

In [11]:
pos_sameby = [
    "Metadata_labels",
]
pos_diffby = ["Metadata_Well"]

neg_sameby = []
neg_diffby = ["Metadata_labels"]

null_size = Pyroptosis_count
batch_size = 1

# number of resampling
n_resamples = 10

### mAP analysis for non-shuffled data

In [12]:
# generate the permutations of cell death labels via itertools
pos_samby_permutations = list(itertools.combinations(df["Metadata_labels"].unique(), 2))

In [13]:
pos_samby_permutations[1]

('Pyroptosis', 'Apoptosis')

In [14]:
results_df = pd.DataFrame(
    columns=[
        "Metadata_Well",
        "Metadata_labels",
        "average_precision",
        "p_value",
        "n_pos_pairs",
        "n_total_pairs",
        "shuffled",
        "comparison",
    ]
)

In [15]:
for i in pos_samby_permutations:
    # print(i)
    tmp = df.copy()
    # get only the rows with the current permutation
    tmp = tmp[tmp["Metadata_labels"].isin(i)]
    # This will generated 100 values [0..100] as seed values
    # This will occur per phenotype

    # spliting metadata and raw feature values
    logging.info("splitting data set into metadata and raw feature values")
    df_meta, df_feats = utils.split_data(df)
    df_feats = np.array(df_feats)

    # execute pipeline on negative control with training dataset with cp features
    try:
        # execute pipeline on negative control with trianing dataset with cp features

        logging.info(f"Running pipeline on CP features using phenotype")
        result = run_pipeline(
            meta=df_meta,
            feats=df_feats,
            pos_sameby=pos_sameby,
            pos_diffby=pos_diffby,
            neg_sameby=neg_sameby,
            neg_diffby=neg_diffby,
            batch_size=batch_size,
            null_size=null_size,
        )

        # adding columns
        result["shuffled"] = "non-shuffled"
        result["comparison"] = "_vs_".join(i)

    except ZeroDivisionError as e:
        logging.warning(f"{e} captured on phenotye:. Skipping")

    # concatenating all datasets
    results_df = pd.concat([results_df, result], ignore_index=True)
    result.to_csv(regular_feat_map_path, index=False)
    result.head()

  0%|          | 0/5348 [00:00<?, ?it/s]

  0%|          | 0/6433 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_22560/362100143.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, result], ignore_index=True)


  0%|          | 0/5348 [00:00<?, ?it/s]

  0%|          | 0/6433 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5348 [00:00<?, ?it/s]

  0%|          | 0/6433 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

### mAP analysis for shuffled data (Feature space)

In [16]:
results_df = pd.DataFrame(
    columns=[
        "Metadata_Well",
        "Metadata_labels",
        "average_precision",
        "p_value",
        "n_pos_pairs",
        "n_total_pairs",
        "shuffled",
        "comparison",
    ]
)

In [17]:
for i in pos_samby_permutations:
    # print(i)
    tmp = df.copy()
    # get only the rows with the current permutation
    tmp = tmp[tmp["Metadata_labels"].isin(i)]
    # This will generated 100 values [0..100] as seed values
    # This will occur per phenotype

    # spliting metadata and raw feature values
    seed = 0

    # split the shuffled dataset
    # spliting metadata and raw feature values
    logging.info("splitting shuffled data set into metadata and raw feature values")
    (
        df_meta,
        df_feats,
    ) = utils.split_data(df)

    df_feats = np.array(df_feats)

    # shuffling the features, this will overwrite the generated feature space from above with the shuffled one
    df_feats = shuffle_features(feature_vals=df_feats, seed=seed)

    try:
        # execute pipeline on negative control with trianing dataset with cp features
        logging.info(
            f"Running pipeline on CP features using phenotype, feature space is shuffled"
        )
        shuffled_feat_map = run_pipeline(
            meta=df_meta,
            feats=df_feats,
            pos_sameby=pos_sameby,
            pos_diffby=pos_diffby,
            neg_sameby=neg_sameby,
            neg_diffby=neg_diffby,
            batch_size=batch_size,
            null_size=null_size,
        )

        # adding shuffle label column
        shuffled_feat_map["shuffled"] = "shuffled_baseline"
        shuffled_feat_map["comparison"] = "_vs_".join(i)

    except ZeroDivisionError as e:
        logging.warning(f"{e} captured on phenotype:  Skipping")

    # concatenating all datasets
    results_df = pd.concat([results_df, shuffled_feat_map], ignore_index=True)
    # saving to csv
    results_df.to_csv(shuffled_feat_space_map_path, index=False)
results_df

  0%|          | 0/5348 [00:00<?, ?it/s]

  0%|          | 0/6433 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_22560/1924883967.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, shuffled_feat_map], ignore_index=True)


  0%|          | 0/5348 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5348 [00:00<?, ?it/s]

  0%|          | 0/6433 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

,Metadata_Well,Metadata_labels,average_precision,p_value,n_pos_pairs,n_total_pairs,shuffled,comparison
0,B02,Pyroptosis,0.445783,0.393939,64,153,shuffled_baseline,Pyroptosis_vs_Control
1,B03,Pyroptosis,0.392328,0.848485,64,153,shuffled_baseline,Pyroptosis_vs_Control
2,B04,Pyroptosis,0.393081,0.848485,64,153,shuffled_baseline,Pyroptosis_vs_Control
3,B05,Pyroptosis,0.419327,0.636364,64,153,shuffled_baseline,Pyroptosis_vs_Control
4,B06,Control,0.519952,0.666667,80,153,shuffled_baseline,Pyroptosis_vs_Control
...,...,...,...,...,...,...,...,...
457,O08,Control,0.522033,0.636364,80,153,shuffled_baseline,Control_vs_Apoptosis
458,O09,Control,0.532334,0.606061,80,153,shuffled_baseline,Control_vs_Apoptosis
459,O10,Control,0.516417,0.696970,80,153,shuffled_baseline,Control_vs_Apoptosis
460,O11,Control,0.540210,0.484848,80,153,shuffled_baseline,Control_vs_Apoptosis
